In [1]:
# import the necessary packages
from imutils.perspective import four_point_transform
from imutils import contours
import imutils
import cv2
import pytesseract
from pytesseract import Output
import json

from sklearn.externals import joblib
from skimage.feature import hog
import numpy as np
from sklearn.cluster import KMeans

from wide_resnet_28_10 import WideResNet28_10
from utils import load_mnist
import matplotlib.pyplot as plt
np.set_printoptions(precision=2)

import os

Using TensorFlow backend.


In [2]:
PATH = './models/'
model_name = "WideResNet28_10"
model=WideResNet28_10()
model.compile()
print('Loading pretrained weights for ', model_name, '...', sep='')
model.load_weights(PATH + model_name + '.h5')

Loading pretrained weights for WideResNet28_10...


In [3]:
def GetIndex(d,word,first=True):
    if word in d['text']:
        if first:
            return d['text'].index(word)
        else:
            return len(d['text']) - 1 - d['text'][::-1].index(word)
    else:
        return -1

In [4]:
def ExtractData(img):
    d = pytesseract.image_to_data(img, output_type=Output.DICT, config="-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz -psm 6")
    return d

In [5]:
def GetBoundingBox(d):
    first_w='SIGLAS'
    last_w='Municipal'
    last_pad=(0,-130)
    padd=100
    p1d=(380,720)#(300,640)
    p2d=(600,1950)
    
    idx_f=GetIndex(d,first_w,True)
    idx_l=GetIndex(d,last_w,False)

    if idx_f!=-1:
        p1= p1d #(d['left'][idx_f]+padd, d['top'][idx_f])
    else:
        
        p1= p1d#default value
        
    if idx_l!=-1:
        #p2=(d['left'][idx_l]+d['width'][idx_l]+last_pad[0]//3, d['top'][idx_l]+d['height'][idx_l]+last_pad[1])
        p2=p2d #(d['left'][idx_l]+d['width'][idx_l]+last_pad[0]//3, d['top'][idx_l]+d['height'][idx_l]+last_pad[1])
    else:
        p2=p2d #default value
    
    if p2[0]-p1[0]<20 or p2[1]-p1[1]<20:
        p2=p2d
        p1= p1d
    
    return (p1,p2)
    
    
    #n_boxes = len(d['level'])
    #for i in range(n_boxes):
    #    if i == 584:            
    #        (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
    #        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)



In [6]:
def DrawBoxes(img,d):
    img2=img.copy()
    n_boxes = len(d['level'])
    for i in range(n_boxes):
            (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
            cv2.rectangle(img2, (x, y), (x + w, y + h), (0, 255, 0), 2)
    return img2

In [7]:
def ShowImg(image):
    cv2.namedWindow('image',cv2.WINDOW_NORMAL)
    cv2.imshow('image',image)
    k = cv2.waitKey(100000)
    if k == 27 or cv2.getWindowProperty('image',cv2.WND_PROP_VISIBLE) < 1:         # wait for ESC key to exit
        cv2.destroyAllWindows()
    cv2.destroyAllWindows()

In [8]:
def get_spaced_colors(n):
    max_value = 16581375 #255**3
    interval = int(max_value / n)
    colors = [hex(I)[2:].zfill(6) for I in range(0, max_value, interval)]
    
    return [(int(i[:2], 16), int(i[2:4], 16), int(i[4:], 16)) for i in colors]

In [9]:
def ExtractRectangles(image):
    # Find contours in the image
    ctrs, hier = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Get rectangles contains each contour
    rects = [cv2.boundingRect(ctr) for ctr in ctrs]
    return rects

In [10]:
def CleanRectangles(rects,thresh=25, thresh2=4):
    cleaned_rects=[]
    cleaned_rects_vcenter=[]
    cleaned_rects_hcenter=[]

    for rect in rects:
        # Draw the rectangles
        if (rect[3]>thresh) and (rect[2]>thresh2):
            cleaned_rects.append(rect)
            cleaned_rects_vcenter.append(rect[1] + rect[3]//2)
            cleaned_rects_hcenter.append(rect[0] + rect[2]//2)
    cleaned_rects_vcenter=np.array(cleaned_rects_vcenter).reshape(-1,1)
    cleaned_rects_hcenter=np.array(cleaned_rects_hcenter).reshape(-1,1)
    return (cleaned_rects,cleaned_rects_vcenter,cleaned_rects_hcenter)

In [11]:
def GetLabels(data,nclust):
    labels=[]  
    kmeans = KMeans(n_clusters=nclust, random_state=0).fit(data)
    means=kmeans.cluster_centers_.mean(axis=1)
    idx = np.argsort(means)
    lut = np.zeros_like(idx)
    lut[idx] = np.arange(nclust)
    for i in range(len(kmeans.labels_)):
        labels.append(lut[kmeans.labels_[i]])
    return (labels,np.sort(means))
#    kparty.labels_[i-1]=lut[i-1]+1b

In [12]:
def AbrirDatosMesa(fname):
    with open(fname) as json_file:  
        data = json.load(json_file)
    return data
        

In [13]:
def CrearCuentasPresidente(mesa,path="./data/"):
    data_name="mesa_"+'{0:06d}'.format(mesa)+'.json'
    data=AbrirDatosMesa(path+data_name)
    
    for eleccion in data['TE']:
        if eleccion['TIPOELECCION'] == '1':
            data_pres=eleccion
    validos=int(data_pres['VOTOSVALIDOS'])
    votos=data_pres['VOTOSTE0']
    cuenta_votos=np.zeros(len(partidos)).astype('int')
    for val in votos:
        name=val['D']
        idx=partidos.index(name)
        v=int(val['V'])
        cuenta_votos[idx]=v
    
    if (np.sum(cuenta_votos)!=validos):
        print("### LOS DATOS EN LA MESA "+str(mesa)+" ESTAN MALOS")
    return cuenta_votos

In [14]:
def PredictLabels(im_final,im_th,cleaned_rects,party_lab,digit_lab,votos_sys):
    bad=False
    tot=0
    rect_pred=np.zeros(len(cleaned_rects)).astype(int)
    vote_to_rect={}
    bad_rect=[]
    party_counts= np.zeros_like(party_lab)
    #Predict labels
    colors=get_spaced_colors(len(votos_sys)-1)
    for i,rect in enumerate(cleaned_rects):
         
        # Make the rectangular region around the digit
        leng = int(rect[3] * 1.2)
        pt1 = int(rect[1] + rect[3] // 2 - leng // 2)
        pt2 = int(rect[0] + rect[2] // 2 - leng // 2)
        if pt1 < 0:
            pt1=0
        if pt2 < 0:
            pt2=0
        roi = im_th[pt1:pt1+leng, pt2:pt2+leng]

        #Resize the image
        roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
        roi = cv2.dilate(roi, (3, 3))
        roi = cv2.erode(roi, (6, 6))
        roi = roi / 255

        ext_digit = roi.reshape(1,28,28,1)
        prediction= model.predict(ext_digit, verbose = 0)
        val = np.argmax(prediction[0])
        
        
        nbr = [val]
        
        mult=1
        
        sbe=str(votos_sys[party_lab[i]])
        if digit_lab[i]==0:
            mult=100
            if(len(sbe)>=3):
                sv=int(sbe[-3])
            else:
                sv=0
        elif digit_lab[i]==1:
            if(len(sbe)>=2):
                sv=int(sbe[-2])
            else:
                sv=0
        else:
            if(len(sbe)>=1):
                sv=int(sbe[-1])
            else:
                sv=0

        party_counts[party_lab[i]]+=val*mult
        rect_pred[i]=val
        if val !=sv:
            #print("IS ",val,"SHOULD BE",sv)
            #pixels = ext_digit.reshape((28, 28))0
            #plt.imshow(pixels, cmap='gray')
            #plt.colorbar()
            #plt.show()
            #print("{} = {}   {}".format((party_lab[i])*3+digit_lab[i],val,prediction)) #,prediction[0])
            bad_rect.append(i)
            bad=True
            tot+=1
 
    for i in bad_rect:
        rect=cleaned_rects[i]
        cv2.rectangle(im_final, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), colors[digit_lab[i]], 3)
        rect=cleaned_rects[i]
        val =rect_pred[i]
        cv2.putText(im_final, str(val), (rect[0], rect[1]),cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)
        #print("HERE")
    return (im_final,party_counts,bad,tot)
    

In [15]:
labels1=['FECHAHORA','NROMESA','TE','N','MESSAGE']
labels2=['FECHAHORA','NROMESA','TE','N','MESSAGE']
partidos=['VAMOS','TODOS','UNE','UNIONISTA','CONVERGENCIA','CREO','FCN-NACION','WINAQ','PPT','UNIDOS','EG','URNG MAIZ','VICTORIA','PHG','VIVA','AVANZA','LIBRE','PAN-PODEMOS','MLP']

In [20]:
def ProcessFile(mesa):
    boleta=1
    fname='{0:06d}'.format(mesa*10+boleta)+'.jpg'
    data_name="mesa_"+'{0:06d}'.format(mesa)+'.json'
    path="data/"
    out_path='./results/'
    
    exists = os.path.isfile(path+fname)
    if not exists:
        print("ACTA ",fname," NO EXISTE")
        return (True,-1)
        exists = os.path.isfile(path+data_name)
    if not exists:
        print("DATOS ",data_name," NO EXISTE")
        return (True,-2)
    

    image = cv2.imread(path+fname)

    d=ExtractData(image)
    (p1,p2)=GetBoundingBox(d)

    #Extract the form area
    crop_img = image[p1[1]:p2[1], p1[0]:p2[0]]
    
    
    #Extract the party arealut = np.zeros_like(idx)
    height, width, channels = crop_img.shape
    crop_partidos = crop_img[:,0:width]

    # Convert to grayscale and apply Gaussian filtering
    im_final=crop_partidos.copy()
    im_gray = cv2.cvtColor(crop_partidos, cv2.COLOR_BGR2GRAY)
    #im_gray = cv2.GaussianBlur(im_gray, (5, 5), 0)

    # Threshold the image
    ret, im_th = cv2.threshold(im_gray, 90, 255, cv2.THRESH_BINARY_INV)

    rects=ExtractRectangles(im_th)

    (cleaned_rects,cleaned_rects_vcenter,cleaned_rects_hcenter) = CleanRectangles(rects)

    npart=19
    ndigit=3
    if (len(cleaned_rects_vcenter)>npart):
        party_lab,party_mean=GetLabels(cleaned_rects_vcenter,npart)
    else:
        return (True,-3)
    
    if (len(cleaned_rects_vcenter)>npart):
        digit_lab,digit_mean=GetLabels(cleaned_rects_hcenter,ndigit)
    else:
        return (True,-4)

    upper_b=int(party_mean[0]-30)
    if upper_b <0:
        upper_b=0
    lower_b=int(party_mean[-1]+30)
    if lower_b >crop_partidos.shape[0]:
        lower_b=crop_partidos.shape[0]-1
    left_b=int(digit_mean[0]-30)
    if left_b <0:
        left_b=0
    right_b=int(digit_mean[-1]+30)
    if right_b >crop_partidos.shape[1]:
        right_b=crop_partidos.shape[1]-1

    if (upper_b!=lower_b) and (left_b!=right_b):    
        crop_partidos = crop_partidos[upper_b:lower_b,left_b:right_b]

    # Convert to grayscale and apply Gaussian filtering
    im_final=crop_partidos.copy()
    im_gray = cv2.cvtColor(crop_partidos, cv2.COLOR_BGR2GRAY)
    #im_gray = cv2.GaussianBlur(im_gray, (5, 5), 0)
    # Threshold the image
    ret, im_th = cv2.threshold(im_gray, 90, 255, cv2.THRESH_BINARY_INV)

    rects=ExtractRectangles(im_th)

    (cleaned_rects,cleaned_rects_vcenter,cleaned_rects_hcenter) = CleanRectangles(rects)

    npart=19
    ndigit=3
    if (len(cleaned_rects_vcenter)>npart):
        party_lab,party_mean=GetLabels(cleaned_rects_vcenter,npart)
    else:
        return (True,-5)
    
    if (len(cleaned_rects_vcenter)>npart):
        digit_lab,digit_mean=GetLabels(cleaned_rects_hcenter,ndigit)
    else:
        return (True,-6)

    votos_sys=CrearCuentasPresidente(mesa)

    (im_final,party_counts,bad,tot)=PredictLabels(im_final,im_th,cleaned_rects,party_lab,digit_lab,votos_sys)

    cv2.imwrite(out_path+fname,im_final)
    return (bad,tot)

In [ ]:
tot_mesas=21100
bad_mesa=[]
for mesa in range(1,tot_mesas+1):    
    bad,tot=ProcessFile(mesa)
    print("Mesa",mesa,bad,tot)
    bad_mesa.append((bad,tot))

Mesa 1 True 4
Mesa 2 True 8
Mesa 3 True 9
Mesa 4 True 3
Mesa 5 True 6
Mesa 6 True 5
Mesa 7 True 4
Mesa 8 True 2
Mesa 9 True 4
Mesa 10 True -3
Mesa 11 False 0
Mesa 12 True 5
Mesa 13 True 2
Mesa 14 True 3
Mesa 15 False 0
Mesa 16 True 3
Mesa 17 True 3
Mesa 18 True 5
Mesa 19 True 10
Mesa 20 True 3
Mesa 21 True 2
Mesa 22 True 15
Mesa 23 True 6
Mesa 24 True 5
Mesa 25 False 0
Mesa 26 True 1
Mesa 27 True 6
Mesa 28 True 4
Mesa 29 True 6
Mesa 30 True 1
Mesa 31 True 2
Mesa 32 True 24
Mesa 33 True 25
Mesa 34 True 1
Mesa 35 True 7
Mesa 36 True 15
Mesa 37 True 2
Mesa 38 False 0
Mesa 39 True 5
Mesa 40 True 1
Mesa 41 True 2
Mesa 42 False 0
Mesa 43 True 4
Mesa 44 False 0
Mesa 45 True 22
Mesa 46 True 9
Mesa 47 True 3
Mesa 48 True 22
Mesa 49 True 1
Mesa 50 True 3
Mesa 51 True 1
Mesa 52 True 4
Mesa 53 True 7
Mesa 54 True 14
Mesa 55 True 16
Mesa 56 True 13
Mesa 57 True 4
Mesa 58 True 7
Mesa 59 True 8
Mesa 60 True -3
Mesa 61 True 5
Mesa 62 False 0
Mesa 63 True 4
Mesa 64 True 6
Mesa 65 True 2
Mesa 66 True 1


Mesa 509 True 4
Mesa 510 True 7
Mesa 511 True 1
Mesa 512 True 6
Mesa 513 True 23
Mesa 514 True 14
Mesa 515 True 12
Mesa 516 True 6
Mesa 517 True 10
Mesa 518 True 22
Mesa 519 True 22
Mesa 520 True 1
Mesa 521 True 2
Mesa 522 True 8
Mesa 523 True 2
Mesa 524 True 3
Mesa 525 True 22
Mesa 526 False 0
Mesa 527 False 0
Mesa 528 True 1
Mesa 529 True 1
Mesa 530 True 20
Mesa 531 True 1
Mesa 532 True 2
Mesa 533 True 1
Mesa 534 True 5
Mesa 535 True 1
Mesa 536 True 6
Mesa 537 True 3
Mesa 538 True 1
Mesa 539 True 15
Mesa 540 True 8
Mesa 541 True 3
Mesa 542 False 0
Mesa 543 True 6
Mesa 544 True 2
Mesa 545 True 17
Mesa 546 True 5
Mesa 547 True 10
Mesa 548 False 0
Mesa 549 True 15
Mesa 550 True 1
Mesa 551 True 4
Mesa 552 True 9
Mesa 553 True 3
Mesa 554 True 1
Mesa 555 True 5
Mesa 556 True 20
Mesa 557 True 11
Mesa 558 True 2
Mesa 559 True 10
Mesa 560 True 6
Mesa 561 True 4
Mesa 562 True 3
Mesa 563 False 0
Mesa 564 True 8
Mesa 565 True 6
Mesa 566 True 2
Mesa 567 False 0
Mesa 568 False 0
Mesa 569 False 0
M

Mesa 1009 True 2
Mesa 1010 True 4
Mesa 1011 True 1
Mesa 1012 True 9
Mesa 1013 True 19
Mesa 1014 True 6
Mesa 1015 True 16
Mesa 1016 True 8
Mesa 1017 True 8
Mesa 1018 True 6
Mesa 1019 True 3
Mesa 1020 True 7
Mesa 1021 True 6
Mesa 1022 True 14
Mesa 1023 True 2
Mesa 1024 True 7
Mesa 1025 True 8
Mesa 1026 True 4
Mesa 1027 True 8
Mesa 1028 True 18
Mesa 1029 True 1
Mesa 1030 True 14
Mesa 1031 True 23
Mesa 1032 True 7
Mesa 1033 True 11
Mesa 1034 True 4
Mesa 1035 True 4
Mesa 1036 True 2
Mesa 1037 True 4
Mesa 1038 True 18
Mesa 1039 True 16
Mesa 1040 True 26
Mesa 1041 True 23
Mesa 1042 True 14
Mesa 1043 True 3
Mesa 1044 True 30
Mesa 1045 True -3
Mesa 1046 True 9
Mesa 1047 True 18
Mesa 1048 True 8
Mesa 1049 True 10
Mesa 1050 True 3
Mesa 1051 True 23
Mesa 1052 False 0
Mesa 1053 True 3
Mesa 1054 True 1
Mesa 1055 True 6
Mesa 1056 False 0
Mesa 1057 True 7
Mesa 1058 True -3
Mesa 1059 True 4
Mesa 1060 True 10
Mesa 1061 True 13
Mesa 1062 True 28
Mesa 1063 True 1
Mesa 1064 True 13
Mesa 1065 True 5
Mesa 10

Mesa 1479 True 25
Mesa 1480 True 25
Mesa 1481 True 25
Mesa 1482 True 24
Mesa 1483 True 24
Mesa 1484 True 24
Mesa 1485 True 22
Mesa 1486 True 24
Mesa 1487 True 20
Mesa 1488 True 2
Mesa 1489 True 4
Mesa 1490 True -3
Mesa 1491 True 1
Mesa 1492 True 7
Mesa 1493 True 3
Mesa 1494 True 3
Mesa 1495 True 1
Mesa 1496 True 5
Mesa 1497 True 21
Mesa 1498 True 4
Mesa 1499 True 20
Mesa 1500 True 24
Mesa 1501 True 2
Mesa 1502 True 27
Mesa 1503 True 2
Mesa 1504 True 4
Mesa 1505 True 7
Mesa 1506 True 4
Mesa 1507 True 11
Mesa 1508 True 25
Mesa 1509 True 4
Mesa 1510 False 0
Mesa 1511 True 16
Mesa 1512 True 4
Mesa 1513 False 0
Mesa 1514 True 10
Mesa 1515 True 1
Mesa 1516 True 1
Mesa 1517 True 2
Mesa 1518 True 1
Mesa 1519 True 2
Mesa 1520 True 10
Mesa 1521 True 2
Mesa 1522 True 2
Mesa 1523 True 3
Mesa 1524 True 7
Mesa 1525 False 0
Mesa 1526 True 2
Mesa 1527 False 0
Mesa 1528 True 4
Mesa 1529 True 6
Mesa 1530 True 12
Mesa 1531 True 5
Mesa 1532 True 5
Mesa 1533 True 2
Mesa 1534 True 5
Mesa 1535 True 1
Mesa 15

Mesa 1939 False 0
Mesa 1940 True 1
Mesa 1941 True 13
Mesa 1942 False 0
Mesa 1943 True 2
Mesa 1944 True 1
Mesa 1945 True 4
Mesa 1946 True 7
Mesa 1947 True 7
Mesa 1948 True 9
Mesa 1949 False 0
Mesa 1950 True 3
Mesa 1951 True 15
Mesa 1952 False 0
Mesa 1953 True 14
Mesa 1954 True 2
Mesa 1955 True 9
Mesa 1956 True 6
Mesa 1957 True 2
Mesa 1958 False 0
ACTA  019591.jpg  NO EXISTE
Mesa 1959 True -1
Mesa 1960 True 3
Mesa 1961 True 3
Mesa 1962 True 1
Mesa 1963 True 11
Mesa 1964 True 9
Mesa 1965 True 9
Mesa 1966 False 0
Mesa 1967 True 22
Mesa 1968 True 7
Mesa 1969 True 5
Mesa 1970 True 7
Mesa 1971 True 16
Mesa 1972 True 22
Mesa 1973 True 7
Mesa 1974 True 2
Mesa 1975 True 8
Mesa 1976 True 4
Mesa 1977 True 19
Mesa 1978 True 9
Mesa 1979 True 1
Mesa 1980 True 17
Mesa 1981 True 4
Mesa 1982 True 7
Mesa 1983 True 9
Mesa 1984 True 6
Mesa 1985 True 7
Mesa 1986 True 14
Mesa 1987 True 4
Mesa 1988 True 6
Mesa 1989 True 7
Mesa 1990 False 0
Mesa 1991 True 6
Mesa 1992 True 12
Mesa 1993 True 1
Mesa 1994 True 5
M

...

Mesa 4600 True 10
Mesa 4601 False 0
Mesa 4602 True 4
Mesa 4603 True 9
Mesa 4604 True 7
Mesa 4605 True 4
Mesa 4606 True 3
Mesa 4607 True 3
Mesa 4608 True 9
Mesa 4609 False 0
Mesa 4610 True 4
Mesa 4611 True 3
Mesa 4612 True 3
Mesa 4613 True 5
Mesa 4614 True 1
Mesa 4615 True 5
Mesa 4616 True 2
Mesa 4617 True 8
Mesa 4618 True 5
Mesa 4619 True 13
Mesa 4620 True 2
Mesa 4621 False 0
ACTA  046221.jpg  NO EXISTE
Mesa 4622 True -1
ACTA  046231.jpg  NO EXISTE
Mesa 4623 True -1
ACTA  046241.jpg  NO EXISTE
Mesa 4624 True -1
ACTA  046251.jpg  NO EXISTE
Mesa 4625 True -1
ACTA  046261.jpg  NO EXISTE
Mesa 4626 True -1
Mesa 4627 True 3
Mesa 4628 True 6
Mesa 4629 True 5
Mesa 4630 True 5
Mesa 4631 False 0
Mesa 4632 True 4
Mesa 4633 True 6
Mesa 4634 True 8
Mesa 4635 True 3
Mesa 4636 True 1
Mesa 4637 True 5
Mesa 4638 True 5
Mesa 4639 True 11
Mesa 4640 True 4
Mesa 4641 True 15
Mesa 4642 True 7
Mesa 4643 True 6
Mesa 4644 True 5
Mesa 4645 True 19
Mesa 4646 True 25
Mesa 4647 True 5
Mesa 4648 False 0
Mesa 4649 T

Mesa 5024 True 5
Mesa 5025 True 7
Mesa 5026 True 3
Mesa 5027 True 12
Mesa 5028 True 2
Mesa 5029 True 8
Mesa 5030 True 7
Mesa 5031 True 8
Mesa 5032 True 5
Mesa 5033 True 16
Mesa 5034 True 6
Mesa 5035 True 7
Mesa 5036 True 21
Mesa 5037 True 2
Mesa 5038 True 4
Mesa 5039 False 0
Mesa 5040 True 6
Mesa 5041 False 0
Mesa 5042 True 4
Mesa 5043 True 7
Mesa 5044 True 2
Mesa 5045 False 0
Mesa 5046 True 4
Mesa 5047 True 6
Mesa 5048 True 1
Mesa 5049 True -3
Mesa 5050 True 7
Mesa 5051 True 13
Mesa 5052 True 7
Mesa 5053 True 4
Mesa 5054 True 4
Mesa 5055 True 1
Mesa 5056 True 3
Mesa 5057 True 3
Mesa 5058 True 4
Mesa 5059 False 0
Mesa 5060 False 0
Mesa 5061 True 10
Mesa 5062 True 6
Mesa 5063 True 1
Mesa 5064 True 2
Mesa 5065 False 0
Mesa 5066 False 0
Mesa 5067 True 4
Mesa 5068 True 1
Mesa 5069 True 5
Mesa 5070 True 1
Mesa 5071 True -3
Mesa 5072 True 6
Mesa 5073 True 8
Mesa 5074 True 5
Mesa 5075 False 0
Mesa 5076 True 1
Mesa 5077 True 6
Mesa 5078 True 17
Mesa 5079 False 0
Mesa 5080 True 6
Mesa 5081 Fals

/home/leo/.local/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (16) found smaller than n_clusters (19). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/leo/.local/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (14) found smaller than n_clusters (19). Possibly due to duplicate points in X.
  return_n_iter=True)


Mesa 5376 True 19
Mesa 5377 False 0
Mesa 5378 True 21
Mesa 5379 True 1
Mesa 5380 True 10
Mesa 5381 True 2
Mesa 5382 True 8
Mesa 5383 True 5
Mesa 5384 True 1
Mesa 5385 True 4
Mesa 5386 True -3
Mesa 5387 True 2
Mesa 5388 True 10
Mesa 5389 True 1
Mesa 5390 True 3
Mesa 5391 True 3
Mesa 5392 True 10
Mesa 5393 True 29
Mesa 5394 True 4
Mesa 5395 True 7
Mesa 5396 True 1
Mesa 5397 True 1
Mesa 5398 True 12
Mesa 5399 True 23
Mesa 5400 False 0
Mesa 5401 True 14
Mesa 5402 False 0
Mesa 5403 True 1
Mesa 5404 True 11
Mesa 5405 True 2
Mesa 5406 True 2
Mesa 5407 False 0
Mesa 5408 True 13
Mesa 5409 True 15
Mesa 5410 True -3
Mesa 5411 False 0
Mesa 5412 True 10
Mesa 5413 False 0
Mesa 5414 True 6
Mesa 5415 True 4
Mesa 5416 True 2
Mesa 5417 False 0
Mesa 5418 True 5
Mesa 5419 True 15
Mesa 5420 True 1
Mesa 5421 True 8
Mesa 5422 True 3
Mesa 5423 True 5
Mesa 5424 True 10
Mesa 5425 True 10
Mesa 5426 True 9
Mesa 5427 True 3
Mesa 5428 True 7
Mesa 5429 True 7
Mesa 5430 True 11
Mesa 5431 True 7
Mesa 5432 True 18
Mesa

Mesa 5825 True 2
Mesa 5826 True 1
Mesa 5827 True 3
Mesa 5828 True 4
Mesa 5829 True 1
Mesa 5830 False 0
Mesa 5831 True 6
Mesa 5832 True 4
Mesa 5833 True 5
Mesa 5834 True 1
Mesa 5835 True 5
Mesa 5836 True 7
Mesa 5837 True 5
Mesa 5838 True 6
Mesa 5839 True 12
Mesa 5840 False 0
Mesa 5841 True 43
Mesa 5842 True 4
Mesa 5843 True 2
Mesa 5844 True 2
Mesa 5845 True 1
Mesa 5846 True 7
Mesa 5847 True 4
Mesa 5848 True 4
Mesa 5849 True 2
Mesa 5850 False 0
Mesa 5851 True 11
Mesa 5852 True 36
Mesa 5853 True 4
Mesa 5854 True 9
Mesa 5855 True 6
Mesa 5856 True 3
Mesa 5857 True 7
Mesa 5858 True 11
Mesa 5859 True 4
Mesa 5860 True 20
Mesa 5861 True 1
Mesa 5862 True 23
Mesa 5863 True 1
Mesa 5864 True 3
Mesa 5865 True 1
Mesa 5866 False 0
Mesa 5867 False 0
Mesa 5868 True 11
Mesa 5869 True 2
Mesa 5870 True 6
Mesa 5871 True 4
Mesa 5872 True 6
Mesa 5873 True 1
Mesa 5874 True 2
Mesa 5875 True 8
Mesa 5876 True 3
Mesa 5877 True 9
Mesa 5878 True 10
Mesa 5879 True 3
Mesa 5880 True 5
Mesa 5881 True 3
Mesa 5882 True 5


In [ ]:
bad_mesa

In [ ]:
mesa=1
boleta=1
fname='{0:06d}'.format(mesa*10+boleta)+'.jpg'
data_name="mesa_"+'{0:06d}'.format(mesa)+'.json'
path="data/"
out_path='./results/'

exists = os.path.isfile(path+fname)

image = cv2.imread(path+fname)

d=ExtractData(image)
(p1,p2)=GetBoundingBox(d)

#Extract the form area
crop_img = image[p1[1]:p2[1], p1[0]:p2[0]]

ShowImg(crop_img)